# 🤖 Confluence RAG Chatbot
### End-to-End Conversational AI with Retrieval-Augmented Generation

---

## 1. Project Overview

This notebook builds a **production-ready conversational chatbot** that answers user questions about Confluence page data using **Retrieval-Augmented Generation (RAG)**.

### Architecture
```
Confluence Pages → LangChain Loader → Text Splitter → OpenAI Embeddings → ChromaDB
                                                                              ↓
User Query → Retriever (top_k) → Prompt Template (strict grounding) → GPT-3.5 Turbo → Answer + Citations
                                                                              ↓
                                                              Gradio Chat UI (memory, clear, sources)
```

### Tech Stack
| Component | Technology |
|---|---|
| Document Loading | `langchain_community.document_loaders.ConfluenceLoader` |
| Text Splitting | `RecursiveCharacterTextSplitter` |
| Embeddings | `OpenAIEmbeddings` (text-embedding-ada-002) |
| Vector Store | `ChromaDB` (in-memory) |
| LLM | `GPT-3.5 Turbo` (temperature 0.1) |
| Chain Pattern | LCEL / Runnable chains (modern LangChain) |
| UI | Gradio Blocks with chat interface |

### Key Features
- Strict grounded answering — only answers from retrieved context
- Citations with page titles and source metadata
- Conversational memory per session
- Adjustable `top_k` retrieval via slider
- Clear Chat button
- Robust error handling throughout

---

## How to Run Locally

### Prerequisites
- Python 3.9+
- An OpenAI API key
- Confluence instance URL + credentials (API token or personal access token)

### Steps

1. **Clone or download** this notebook to your local machine.

2. **Create a virtual environment** (recommended):
   ```bash
   python -m venv venv
   source venv/bin/activate  # macOS/Linux
   venv\Scripts\activate     # Windows
   ```

3. **Install dependencies** — Run the first code cell (Section 2) which installs all packages.

4. **Set environment variables** for local use (instead of Colab userdata):
   ```bash
   export OPENAI_API_KEY="sk-your-key-here"
   export CONFLUENCE_URL="https://your-domain.atlassian.net/wiki"
   export CONFLUENCE_USERNAME="your-email@company.com"
   export CONFLUENCE_API_KEY="your-confluence-api-token"
   export CONFLUENCE_SPACE_KEY="YOUR_SPACE_KEY"
   ```
   Or create a `.env` file in the same directory with these values.

5. **Run all cells** sequentially in Jupyter Lab / Notebook:
   ```bash
   jupyter notebook confluence_rag_chatbot.ipynb
   ```

6. **Access the Gradio UI** — The last cell launches a Gradio interface. Open the local URL printed in the output (typically `http://127.0.0.1:7860`).

> **Note for Colab users**: The notebook auto-detects Colab and uses `google.colab.userdata` for secrets. Set your secrets in Colab's sidebar (🔑 icon).

---

## 2. Setup: Install Dependencies & Imports

In [ ]:
# ============================================================
# 2a. Install all required packages
# ============================================================
# Run this cell once — it installs everything needed.

import subprocess
import sys

def install(package):
    """Install a package using pip, suppressing verbose output."""
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "-q", package],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

packages = [
    "langchain>=0.3.0",
    "langchain-community>=0.3.0",
    "langchain-openai>=0.2.0",
    "langchain-chroma>=0.2.0",
    "chromadb>=0.5.0",
    "atlassian-python-api>=3.41.0",
    "beautifulsoup4>=4.12.0",
    "lxml>=5.0.0",
    "gradio>=4.0.0",
    "python-dotenv>=1.0.0",
]

print("📦 Installing dependencies...")
for pkg in packages:
    try:
        install(pkg)
        print(f"  ✅ {pkg}")
    except Exception as e:
        print(f"  ❌ Failed to install {pkg}: {e}")

print("\n✅ All dependencies installed successfully!")

In [ ]:
# ============================================================
# 2b. Import all modules
# ============================================================

import os
import warnings
import textwrap
from typing import List, Dict, Any, Optional, Tuple

warnings.filterwarnings("ignore")

# --- LangChain Core ---
from langchain_community.document_loaders import ConfluenceLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.documents import Document

# --- Gradio ---
import gradio as gr

# --- Utilities ---
from dotenv import load_dotenv

print("✅ All modules imported successfully!")

---

## 3. OpenAI API Setup

The API key is loaded using the following priority:
1. **Google Colab** `userdata.get('OPENAI_API_KEY')` (if running in Colab)
2. **Environment variable** `OPENAI_API_KEY`
3. **`.env` file** in the current directory

In [ ]:
# ============================================================
# 3. Load API Keys (OpenAI + Confluence)
# ============================================================

def get_secret(key_name: str, required: bool = True) -> Optional[str]:
    """
    Retrieve a secret with fallback chain:
      1. Google Colab userdata
      2. OS environment variable
      3. .env file
    """
    value = None

    # 1) Try Colab userdata
    try:
        from google.colab import userdata
        value = userdata.get(key_name)
        if value:
            print(f"  🔑 {key_name}: loaded from Colab userdata")
            return value
    except (ImportError, Exception):
        pass

    # 2) Try OS environment
    value = os.environ.get(key_name)
    if value:
        print(f"  🔑 {key_name}: loaded from environment variable")
        return value

    # 3) Try .env file
    load_dotenv()
    value = os.environ.get(key_name)
    if value:
        print(f"  🔑 {key_name}: loaded from .env file")
        return value

    if required:
        raise ValueError(
            f"❌ {key_name} not found! Set it via:\n"
            f"   - Colab: Sidebar → 🔑 Secrets → Add '{key_name}'\n"
            f"   - Local: export {key_name}='your-key' or add to .env file"
        )
    return None


print("Loading secrets...\n")

# --- OpenAI ---
OPENAI_API_KEY = get_secret("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Confluence ---
CONFLUENCE_URL = get_secret("CONFLUENCE_URL")
CONFLUENCE_USERNAME = get_secret("CONFLUENCE_USERNAME", required=False)
CONFLUENCE_API_KEY = get_secret("CONFLUENCE_API_KEY", required=False)
CONFLUENCE_SPACE_KEY = get_secret("CONFLUENCE_SPACE_KEY")

# Optional: Personal Access Token (for Confluence Server/Data Center)
CONFLUENCE_PAT = get_secret("CONFLUENCE_PAT", required=False)

# --- Validate OpenAI key ---
try:
    test_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=5)
    test_llm.invoke("test")
    print("\n✅ OpenAI API key is valid and working!")
except Exception as e:
    print(f"\n❌ OpenAI API key validation failed: {e}")
    raise

---

## 4. Load Confluence Pages

We use `ConfluenceLoader` from `langchain_community.document_loaders` to load pages from your Confluence space. This loader:

- Connects to Confluence via REST API
- Recursively loads **all pages including sub-pages** in the specified space
- Extracts text content and preserves metadata (page title, URL, page ID, etc.)
- Handles nested page hierarchies automatically

### Authentication Methods Supported
1. **Confluence Cloud** (Atlassian): `username` + `api_key` (API token)
2. **Confluence Server/Data Center**: `token` (Personal Access Token)

In [ ]:
# ============================================================
# 4. Load Confluence Pages (including sub-pages)
# ============================================================

def load_confluence_documents(
    url: str,
    space_key: str,
    username: Optional[str] = None,
    api_key: Optional[str] = None,
    token: Optional[str] = None,
    max_pages: int = 250,
    include_attachments: bool = False,
) -> List[Document]:
    """
    Load all pages (and sub-pages) from a Confluence space.

    Args:
        url: Confluence base URL (e.g., https://mycompany.atlassian.net/wiki)
        space_key: The Confluence space key (e.g., "HR", "ENG", "KB")
        username: Email for Confluence Cloud auth
        api_key: API token for Confluence Cloud auth
        token: Personal Access Token for Confluence Server/DC
        max_pages: Maximum number of pages to load
        include_attachments: Whether to load file attachments

    Returns:
        List of LangChain Document objects
    """
    print(f"🔄 Connecting to Confluence: {url}")
    print(f"   Space: {space_key}")
    print(f"   Max pages: {max_pages}")
    print(f"   Include attachments: {include_attachments}")
    print()

    try:
        # Build loader kwargs based on authentication method
        loader_kwargs = {
            "url": url,
            "space_key": space_key,
            "include_attachments": include_attachments,
            "limit": 50,  # Pages per API request (pagination batch size)
            "max_pages": max_pages,
            "keep_markdown_format": True,  # Preserve formatting
            "content_format": "storage",   # Get full HTML storage format
        }

        # Authentication: Cloud (username + api_key) vs Server (PAT)
        if token:
            loader_kwargs["token"] = token
            print("   Auth: Personal Access Token (Server/DC)")
        elif username and api_key:
            loader_kwargs["username"] = username
            loader_kwargs["api_key"] = api_key
            print("   Auth: Username + API Token (Cloud)")
        else:
            raise ValueError(
                "❌ No valid Confluence credentials provided!\n"
                "   Cloud: Set CONFLUENCE_USERNAME and CONFLUENCE_API_KEY\n"
                "   Server/DC: Set CONFLUENCE_PAT"
            )

        # Initialize and load
        loader = ConfluenceLoader(**loader_kwargs)
        documents = loader.load()

        if not documents:
            raise ValueError(
                f"❌ No documents loaded from space '{space_key}'.\n"
                f"   Check that the space key is correct and you have read access."
            )

        # Enrich metadata
        for i, doc in enumerate(documents):
            doc.metadata["chunk_index"] = i
            doc.metadata["source_type"] = "confluence"
            # Ensure page_title is available
            if "title" in doc.metadata:
                doc.metadata["page_title"] = doc.metadata["title"]
            elif "source" in doc.metadata:
                doc.metadata["page_title"] = doc.metadata["source"]
            else:
                doc.metadata["page_title"] = f"Page {i+1}"

        print(f"\n✅ Successfully loaded {len(documents)} Confluence pages!")
        print(f"\n📄 Pages loaded:")
        for i, doc in enumerate(documents[:20]):  # Show first 20
            title = doc.metadata.get("page_title", "Untitled")
            content_len = len(doc.page_content)
            print(f"   {i+1:3d}. {title} ({content_len:,} chars)")
        if len(documents) > 20:
            print(f"   ... and {len(documents) - 20} more pages")

        return documents

    except Exception as e:
        print(f"\n❌ Error loading Confluence pages: {e}")
        raise


# --- Load documents ---
raw_documents = load_confluence_documents(
    url=CONFLUENCE_URL,
    space_key=CONFLUENCE_SPACE_KEY,
    username=CONFLUENCE_USERNAME,
    api_key=CONFLUENCE_API_KEY,
    token=CONFLUENCE_PAT,
    max_pages=250,
    include_attachments=False,
)

# Show sample content
if raw_documents:
    print(f"\n{'='*60}")
    print(f"📋 Sample content from first page:")
    print(f"{'='*60}")
    print(f"Title: {raw_documents[0].metadata.get('page_title', 'N/A')}")
    print(f"Content preview (first 500 chars):\n")
    print(raw_documents[0].page_content[:500])

---

## 5. Chunking / Text Splitting

### Why chunk?
Confluence pages can be very long. Embedding entire pages reduces retrieval precision — the embedding of a 10,000-character page is a blurry average of many topics. Chunking creates focused, semantically coherent pieces that match specific queries better.

### Parameter Choices

| Parameter | Value | Rationale |
|---|---|---|
| `chunk_size` | **1000** | Balances semantic completeness with retrieval precision. Large enough to capture a full paragraph/concept, small enough for targeted retrieval. |
| `chunk_overlap` | **200** | 20% overlap ensures context continuity — sentences at chunk boundaries aren't lost. Prevents splitting mid-thought. |
| `separators` | `["\n\n", "\n", ". ", " "]` | Splits on natural boundaries: paragraphs → lines → sentences → words. Preserves document structure. |

In [ ]:
# ============================================================
# 5. Text Splitting
# ============================================================

CHUNK_SIZE = 1000       # Characters per chunk
CHUNK_OVERLAP = 200     # Overlap between consecutive chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
    is_separator_regex=False,
)

# Filter out empty or near-empty documents before splitting
valid_documents = [
    doc for doc in raw_documents
    if doc.page_content and len(doc.page_content.strip()) > 50
]
print(f"📄 Valid documents (non-empty, >50 chars): {len(valid_documents)} / {len(raw_documents)}")

# Split into chunks
chunks = text_splitter.split_documents(valid_documents)

# Add chunk-level metadata for citation
for i, chunk in enumerate(chunks):
    chunk.metadata["chunk_id"] = f"chunk_{i:04d}"
    chunk.metadata["chunk_char_count"] = len(chunk.page_content)

print(f"\n✅ Text splitting complete!")
print(f"   📊 Total chunks created: {len(chunks)}")
print(f"   📏 Chunk size: {CHUNK_SIZE} chars")
print(f"   🔗 Chunk overlap: {CHUNK_OVERLAP} chars")

# Distribution stats
chunk_lengths = [len(c.page_content) for c in chunks]
print(f"\n   📈 Chunk length statistics:")
print(f"      Min: {min(chunk_lengths):,} chars")
print(f"      Max: {max(chunk_lengths):,} chars")
print(f"      Avg: {sum(chunk_lengths)/len(chunk_lengths):,.0f} chars")

# Show sample chunk
print(f"\n{'='*60}")
print(f"📋 Sample chunk (chunk_0000):")
print(f"{'='*60}")
print(f"Page: {chunks[0].metadata.get('page_title', 'N/A')}")
print(f"Chunk ID: {chunks[0].metadata.get('chunk_id', 'N/A')}")
print(f"Content:\n{chunks[0].page_content[:400]}...")

---

## 6. Embeddings + ChromaDB Vector Store

We use OpenAI's `text-embedding-ada-002` model via `OpenAIEmbeddings` to convert text chunks into dense vectors, then store them in an **in-memory ChromaDB** instance for fast similarity search.

In [ ]:
# ============================================================
# 6. Create Embeddings & ChromaDB Vector Store
# ============================================================

print("🔄 Creating embeddings and building vector store...")
print(f"   Model: text-embedding-ada-002")
print(f"   Chunks to embed: {len(chunks)}")
print()

try:
    # Initialize embeddings
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002",
        openai_api_key=OPENAI_API_KEY,
    )

    # Build ChromaDB vector store (in-memory)
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        collection_name="confluence_rag",
        collection_metadata={"hnsw:space": "cosine"},  # Cosine similarity
    )

    print(f"✅ Vector store created successfully!")
    print(f"   📊 Total vectors stored: {vectorstore._collection.count()}")
    print(f"   💾 Storage: In-memory (ChromaDB)")
    print(f"   📐 Distance metric: Cosine similarity")

    # Quick test retrieval
    test_results = vectorstore.similarity_search("test", k=1)
    if test_results:
        print(f"\n   🧪 Test retrieval: OK (retrieved {len(test_results)} result)")
    else:
        print(f"\n   ⚠️  Test retrieval returned no results.")

except Exception as e:
    print(f"\n❌ Error creating vector store: {e}")
    raise

---

## 7. Retriever + RAG QA Chain (GPT-3.5 Turbo)

We build the RAG chain using **modern LangChain LCEL (LangChain Expression Language)** patterns — no deprecated `ConversationalRetrievalChain` or `RetrievalQA`.

### Chain Architecture
```
User Input → Format chat history → Retrieve relevant chunks (top_k)
     ↓
Strict Prompt Template (system + context + history + question)
     ↓
GPT-3.5 Turbo (temperature=0.1) → Grounded answer with citations
```

---

## 8. Prompt Template (Strict Grounding + Citations)

In [ ]:
# ============================================================
# 7 & 8. LLM, Prompt Template, and RAG Chain
# ============================================================

# --- Initialize LLM ---
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,             # Low temperature for factual accuracy
    max_tokens=1024,
    openai_api_key=OPENAI_API_KEY,
)
print("✅ LLM initialized: gpt-3.5-turbo (temperature=0.1)")


# --- Strict Prompt Template ---
SYSTEM_PROMPT = """You are a helpful, accurate, and concise assistant that answers questions ONLY based on the provided Confluence page context.

## STRICT RULES:
1. **ONLY answer from the provided context.** Do NOT use any prior knowledge or make assumptions.
2. **If the answer is not in the context, say exactly:** "I don't know based on the provided document."
3. **Always cite your sources** using the page title and chunk ID from the metadata.
4. **Keep answers concise** — aim for 2-5 sentences unless a detailed answer is clearly needed.
5. **Be compliance-friendly** — do not speculate, do not provide personal opinions.
6. **Format citations** at the end of your answer like:
   📖 Sources:
   - [Page Title] (Chunk: chunk_XXXX)

## CONTEXT FROM CONFLUENCE PAGES:
{context}

## CONVERSATION HISTORY:
{chat_history}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{question}"),
])

print("✅ Prompt template created (strict grounding + citations)")


# --- Helper functions ---
def format_docs_with_metadata(docs: List[Document]) -> str:
    """Format retrieved documents with metadata for the prompt context."""
    if not docs:
        return "No relevant documents found."

    formatted = []
    for i, doc in enumerate(docs):
        page_title = doc.metadata.get("page_title", "Unknown Page")
        chunk_id = doc.metadata.get("chunk_id", f"chunk_{i}")
        source_url = doc.metadata.get("source", "")

        header = f"--- Source {i+1}: [{page_title}] (Chunk: {chunk_id}) ---"
        if source_url:
            header += f"\nURL: {source_url}"

        formatted.append(f"{header}\n{doc.page_content}\n")

    return "\n".join(formatted)


def format_chat_history(history: List[Tuple[str, str]]) -> str:
    """Format conversation history for the prompt."""
    if not history:
        return "No previous conversation."

    formatted = []
    for human, ai in history[-5:]:  # Keep last 5 exchanges to manage context window
        formatted.append(f"Human: {human}")
        formatted.append(f"Assistant: {ai}")

    return "\n".join(formatted)


def extract_source_citations(docs: List[Document]) -> str:
    """Extract unique source citations from retrieved documents."""
    seen = set()
    citations = []

    for doc in docs:
        page_title = doc.metadata.get("page_title", "Unknown")
        chunk_id = doc.metadata.get("chunk_id", "N/A")
        source_url = doc.metadata.get("source", "")
        excerpt = doc.page_content[:100].replace("\n", " ").strip()

        key = f"{page_title}_{chunk_id}"
        if key not in seen:
            seen.add(key)
            citation = f"- **{page_title}** (Chunk: {chunk_id})"
            if source_url:
                citation += f" | [Link]({source_url})"
            citation += f"\n  _Excerpt: \"{excerpt}...\"_"
            citations.append(citation)

    return "\n".join(citations) if citations else "No sources available."


# --- Build the RAG chain (LCEL) ---
def create_rag_chain(top_k: int = 4):
    """
    Create a RAG chain using LCEL with configurable top_k.
    Returns: (chain, retriever)
    """
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": top_k},
    )

    chain = (
        {
            "context": lambda x: format_docs_with_metadata(
                retriever.invoke(x["question"])
            ),
            "chat_history": lambda x: format_chat_history(x.get("chat_history", [])),
            "question": lambda x: x["question"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain, retriever


# Create default chain
rag_chain, retriever = create_rag_chain(top_k=4)
print("✅ RAG chain built with LCEL (top_k=4)")


# --- Query function with full error handling ---
def query_rag(
    question: str,
    chat_history: List[Tuple[str, str]] = None,
    top_k: int = 4,
) -> Dict[str, Any]:
    """
    Query the RAG chain with error handling and citation extraction.

    Returns:
        dict with 'answer', 'sources', 'retrieved_docs'
    """
    if chat_history is None:
        chat_history = []

    if not question or not question.strip():
        return {
            "answer": "Please enter a question.",
            "sources": "",
            "retrieved_docs": [],
        }

    try:
        # Recreate chain if top_k changed
        chain, ret = create_rag_chain(top_k=top_k)

        # Retrieve documents for citation
        retrieved_docs = ret.invoke(question)

        # Check for empty retrieval
        if not retrieved_docs:
            return {
                "answer": "I don't know based on the provided document. No relevant content was found in the Confluence pages for your query.",
                "sources": "No matching documents found.",
                "retrieved_docs": [],
            }

        # Run chain
        answer = chain.invoke({
            "question": question,
            "chat_history": chat_history,
        })

        # Extract citations
        sources = extract_source_citations(retrieved_docs)

        return {
            "answer": answer,
            "sources": sources,
            "retrieved_docs": retrieved_docs,
        }

    except Exception as e:
        error_msg = str(e)
        if "rate_limit" in error_msg.lower() or "429" in error_msg:
            return {
                "answer": "⚠️ OpenAI rate limit reached. Please wait a moment and try again.",
                "sources": "",
                "retrieved_docs": [],
            }
        elif "authentication" in error_msg.lower() or "401" in error_msg:
            return {
                "answer": "❌ OpenAI API authentication error. Please check your API key.",
                "sources": "",
                "retrieved_docs": [],
            }
        else:
            return {
                "answer": f"❌ An error occurred: {error_msg}",
                "sources": "",
                "retrieved_docs": [],
            }


print("\n✅ RAG query function ready with full error handling!")

In [ ]:
# ============================================================
# Quick test (before building UI)
# ============================================================

print("🧪 Running a quick test query...\n")

test_result = query_rag("What are the main topics covered in these pages?", top_k=3)

print(f"Question: What are the main topics covered in these pages?")
print(f"\nAnswer:\n{test_result['answer']}")
print(f"\n📖 Sources:\n{test_result['sources']}")
print(f"\n✅ Test complete! Retrieved {len(test_result['retrieved_docs'])} chunks.")

---

## 9. Gradio Chat UI

The Gradio interface provides:
- **Chat-style conversation** with persistent session memory
- **Clear Chat** button to reset conversation
- **top_k slider** to control how many chunks are retrieved (1–10)
- **Sources panel** showing citations with page titles, chunk IDs, and excerpts

In [ ]:
# ============================================================
# 9. Gradio Chat UI
# ============================================================

def respond(
    message: str,
    chat_history: List[List[str]],
    top_k: int,
) -> Tuple[str, List[List[str]], str]:
    """
    Process user message through RAG pipeline and return response.

    Args:
        message: User's question
        chat_history: List of [user_msg, bot_msg] pairs (Gradio format)
        top_k: Number of chunks to retrieve

    Returns:
        (bot_response, updated_history, sources_text)
    """
    if not message.strip():
        return "", chat_history, ""

    # Convert Gradio history format to our tuple format
    history_tuples = [(h[0], h[1]) for h in chat_history if len(h) == 2 and h[1]]

    # Query RAG
    result = query_rag(
        question=message,
        chat_history=history_tuples,
        top_k=top_k,
    )

    answer = result["answer"]
    sources = result["sources"]

    # Update history
    chat_history.append([message, answer])

    # Format sources display
    sources_display = f"### 📖 Sources (top_{top_k} retrieval)\n\n{sources}" if sources else "No sources."

    return "", chat_history, sources_display


def clear_chat():
    """Clear conversation history and sources."""
    return [], "", ""


# --- Build Gradio Interface ---
with gr.Blocks(
    title="📚 Confluence RAG Chatbot",
    theme=gr.themes.Soft(),
    css="""
    .source-box { max-height: 300px; overflow-y: auto; }
    footer { display: none !important; }
    """
) as demo:

    gr.Markdown(
        """
        # 📚 Confluence RAG Chatbot
        ### Ask questions about your Confluence pages — answers are grounded in your documents with citations.

        **Features:** Conversational memory • Adjustable retrieval depth • Source citations
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                label="Conversation",
                height=500,
                show_copy_button=True,
                bubble_full_width=False,
                avatar_images=(None, "https://em-content.zobj.net/source/twitter/376/robot_1f916.png"),
            )

            with gr.Row():
                msg_input = gr.Textbox(
                    placeholder="Ask a question about your Confluence pages...",
                    label="Your Question",
                    scale=4,
                    lines=1,
                    max_lines=3,
                )
                send_btn = gr.Button("Send 📤", variant="primary", scale=1)

            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary")
                top_k_slider = gr.Slider(
                    minimum=1,
                    maximum=10,
                    value=4,
                    step=1,
                    label="Retrieval Depth (top_k)",
                    info="Number of document chunks to retrieve per query",
                )

        with gr.Column(scale=2):
            sources_display = gr.Markdown(
                value="### 📖 Sources\n\n_Ask a question to see source citations here._",
                label="Sources & Citations",
                elem_classes=["source-box"],
            )

            gr.Markdown(
                """
                ---
                ### ℹ️ How to Use
                1. Type your question in the text box
                2. Adjust **top_k** slider for more/fewer source chunks
                3. View citations in the **Sources** panel
                4. Click **Clear Chat** to start a new conversation

                ### 🛡️ Grounding Policy
                - Answers come **only** from loaded Confluence pages
                - If info isn't found, the bot will say so
                - Citations include page title and chunk ID
                """
            )

    # --- Event handlers ---
    send_btn.click(
        fn=respond,
        inputs=[msg_input, chatbot, top_k_slider],
        outputs=[msg_input, chatbot, sources_display],
    )

    msg_input.submit(
        fn=respond,
        inputs=[msg_input, chatbot, top_k_slider],
        outputs=[msg_input, chatbot, sources_display],
    )

    clear_btn.click(
        fn=clear_chat,
        inputs=[],
        outputs=[chatbot, sources_display, msg_input],
    )

print("✅ Gradio UI built successfully!")
print("🚀 Launching...")

# Launch the UI
demo.launch(
    share=False,        # Set True for public Colab link
    debug=False,
    server_name="0.0.0.0",  # Accessible from any interface
    server_port=7860,
)

---

## 10. Demo + Testing

Below are test queries you can run programmatically to validate the pipeline (without the Gradio UI).

> **Note:** These demonstrate expected behavior — actual answers depend on YOUR Confluence content.

In [ ]:
# ============================================================
# 10. Demo / Testing (programmatic, no UI needed)
# ============================================================

# Stop the Gradio server first if running
try:
    demo.close()
except:
    pass

print("="*70)
print("🧪 DEMO: RAG Pipeline Testing")
print("="*70)

# --- Test queries ---
test_questions = [
    "What are the main topics covered in these Confluence pages?",
    "Summarize the key policies or guidelines mentioned.",
    "What is the process described for onboarding?",      # May say "I don't know" if not in pages
    "Tell me about quantum computing advancements.",       # Should say "I don't know" — off-topic
]

chat_history = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'─'*60}")
    print(f"🔹 Test {i}: {question}")
    print(f"{'─'*60}")

    result = query_rag(question, chat_history=chat_history, top_k=4)

    print(f"\n💬 Answer:\n{result['answer']}")
    print(f"\n📖 Sources:\n{result['sources']}")
    print(f"\n📊 Chunks retrieved: {len(result['retrieved_docs'])}")

    # Add to history for conversational continuity
    chat_history.append((question, result['answer']))

print(f"\n{'='*70}")
print(f"✅ Demo complete! {len(test_questions)} queries tested.")
print(f"   Conversation history: {len(chat_history)} exchanges")
print(f"{'='*70}")

# --- Expected behavior ---
print("""
📋 EXPECTED BEHAVIOR:

1. Questions about Confluence content → Grounded answer + citations
   Example: "What is the leave policy?" → Answer from HR pages + page title + chunk ID

2. Off-topic questions → "I don't know based on the provided document."
   Example: "Tell me about quantum computing" → Polite refusal

3. Follow-up questions → Uses conversation history for context
   Example: "Can you elaborate on that?" → References previous answer's topic

4. Empty retrieval → Clear message that no relevant content was found
""")

---

## 11. Troubleshooting / Common Errors

### 🔑 API Key Issues

| Error | Cause | Fix |
|---|---|---|
| `OPENAI_API_KEY not found` | Key not set | **Colab:** Add to Secrets sidebar. **Local:** `export OPENAI_API_KEY='sk-...'` or add to `.env` |
| `AuthenticationError` | Invalid or expired key | Generate a new key at [platform.openai.com](https://platform.openai.com/api-keys) |
| `RateLimitError` | Too many requests | Wait 60 seconds and retry. Consider upgrading your OpenAI plan. |

### 📄 Confluence Loading Issues

| Error | Cause | Fix |
|---|---|---|
| `No documents loaded` | Wrong space key or no access | Verify `CONFLUENCE_SPACE_KEY` and that your credentials have read access |
| `401 Unauthorized` | Bad credentials | **Cloud:** Use email + API token (not password). **Server:** Use Personal Access Token |
| `ConnectionError` | Wrong URL | Ensure URL format: `https://your-domain.atlassian.net/wiki` |
| `Empty pages` | Pages have no text content | Check if pages use macros/embedded content that the loader can't extract |

### 🔧 Module/Import Errors

| Error | Fix |
|---|---|
| `ModuleNotFoundError: langchain_community` | Re-run the install cell (Section 2) |
| `ImportError: cannot import name 'ConfluenceLoader'` | `pip install -q atlassian-python-api lxml beautifulsoup4` |
| `ModuleNotFoundError: langchain_chroma` | `pip install -q langchain-chroma` |

### 💡 Common Runtime Issues

| Issue | Fix |
|---|---|
| Gradio port in use | Change `server_port` in `demo.launch()` or kill process: `lsof -i :7860` |
| Slow embeddings | Normal for first run with many chunks. ChromaDB caches after that. |
| `ChromaDB` warnings | Safely ignorable. Set `warnings.filterwarnings('ignore')` |
| Chat memory too long | The prompt limits history to last 5 exchanges to manage context window |

### 🔄 Confluence Authentication Guide

**For Confluence Cloud (Atlassian):**
1. Go to [id.atlassian.com/manage-profile/security/api-tokens](https://id.atlassian.com/manage-profile/security/api-tokens)
2. Create an API token
3. Set `CONFLUENCE_USERNAME` = your email, `CONFLUENCE_API_KEY` = the token

**For Confluence Server/Data Center:**
1. Go to Profile → Personal Access Tokens
2. Create a new token with read permissions
3. Set `CONFLUENCE_PAT` = the token

### 📧 Still stuck?
Check LangChain docs: [python.langchain.com/docs/integrations/document_loaders/confluence](https://python.langchain.com/docs/integrations/document_loaders/confluence)